In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/pricepred/train.csv")
submit_set = pd.read_csv("/kaggle/input/pricepred/testFeatures.csv")
inflation = pd.read_csv("/kaggle/input/pricepred/inflation.csv")
train.head()

In [ ]:
unique_products = train["ürün"].unique()
unique_dates = train["tarih"].unique()

In [ ]:
inflation.head()

In [ ]:
def find_change(df) -> pd.DataFrame:
    if isinstance(df, pd.DataFrame):
        df = df.reset_index()
        pct_list = list()
        product_list = df["ürün"].unique()
        for i, col in enumerate(product_list):
            pct = df.loc[df["ürün"] == f"{col}"]["ürün fiyatı"].pct_change(periods=1).reset_index()
            pct_list.append(pct)

        pct_df = pd.DataFrame(data=pct_list[0])
        merged = pd.concat(pct_list).rename(columns={"ürün fiyatı": "pct change"})
        a = df[["ürün fiyatı"]].reset_index()
        final_df = pd.merge(nn_train, merged, how="inner", on="index")
        final_df = pd.merge(final_df, df[["tarih", "ürün","index"]],how="inner",on="index")
        final_df = final_df.drop("index", axis=1)
        final_df["pct change"] = final_df["pct change"].fillna(0)
        return final_df
        
    else:
        print("[INFO]:The input you provided is not a pd.DataFrame")

In [ ]:
abc = find_change(train)
abc

In [ ]:
final_df = final_df.drop("index", axis=1)
final_df["pct change"] = final_df["pct change"].fillna(0)
final_df.loc[final_df["ürün"] == "kıyma"][:50]

In [ ]:
final_df["ürün fiyatı"] = final_df["ürün fiyatı"] + (final_df["ürün fiyatı"] * final_df["pct change"] )

In [ ]:
final_df.loc[final_df["ürün"] == "kıyma"].loc[]

In [ ]:
train_len = int(0.8 * len(final_df["ürün fiyatı"]))
train_set, test_set = final_df.iloc[:train_len,], final_df.iloc[train_len:,]
y, x = train_set["ürün fiyatı"], train_set.drop("ürün fiyatı", axis=1)

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
train_set[["tarih", "ürün fiyatı_x"]].rename(columns=({"tarih":"ds", "ürün fiyatı_x":"y"}))

In [ ]:
models = dict()
for product in unique_products:
    model = Prophet()
    prod = train_set.loc[train_set["ürün"] == product]
    prod = prod[["tarih", "ürün fiyatı_x"]].rename(columns=({"tarih":"ds", "ürün fiyatı_x":"y"}))
    model.fit(prod)
    models.update({product: model})

In [ ]:
test = test_set.loc[test_set["ürün"] == "tavuk"].reset_index().sort_values(by="index")
test_date = pd.DataFrame(test["tarih"]).rename(columns={"tarih":"ds"})
test_y = pd.DataFrame(test["ürün fiyatı_x"])
test_date

In [ ]:
ls = list()
for product in unique_products:
    test = test_set.loc[test_set["ürün"] == product].reset_index().sort_values(by="index")
    test_date = pd.DataFrame(test["tarih"]).rename(columns={"tarih":"ds"})
    y = pd.DataFrame(test["ürün fiyatı_x"])
    y_hat = models[product].predict(test_date)["yhat"]
    ls.append([product, y, y_hat])

In [ ]:
from sklearn.metrics import mean_squared_error

error_sum = 0
for product, y, y_hat in ls:
    y_hat = y_hat.values.reshape((len(y_hat), 1))
    y = y.values
    error_sum += mean_squared_error(y, y_hat)

hello_world = error_sum/len(unique_products)
print(len(unique_products))
print(f"ABI HATA:{hello_world}")

In [ ]:
real_test_set = pd.read_csv("/kaggle/input/priceprediction/testFeatures.csv")
real_test_set.head()

In [ ]:
test = real_test_set.loc[real_test_set["ürün"] == "tavuk"]
test_date = pd.DataFrame(test["tarih"]).rename(columns={"tarih":"ds"})


In [ ]:
results = list()
for product in unique_products:
    test = real_test_set.loc[real_test_set["ürün"] == product]
    index = test["id"]
    test_date = pd.DataFrame(test["tarih"]).rename(columns={"tarih":"ds"})
    y_hat = models[product].predict(test_date)["yhat"]
    results.append([product, index, y_hat])

In [ ]:
ids = pd.DataFrame(results[1][1])
ys = pd.DataFrame(results[1][2])
ids = pd.concat([ids, ys], axis=0)
ids

In [ ]:
product_list = list()
for i, product in enumerate(unique_products):
    pass

results = list(results)

In [ ]:
bambaska = list()
for vec in results:
    for i, index in enumerate(vec[1]):
        bambaska.append([index, vec[2][i]])
ohh = sorted(bambaska, key=lambda oy: oy[0])
buson = pd.DataFrame(ohh)
buson = buson.drop([0], axis=1)
buson.to_csv("ssolutions.csv")